In [3]:
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv(r"Crop_recommendation.csv")

In [30]:
df['label'].unique()

array(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee'],
      dtype=object)

In [14]:
df

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,coffee
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,coffee
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,coffee
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,coffee


In [25]:
gf = pd.read_csv("Crop_Data.xlsx.csv")
gf

,temperature,humidity,ph,rainfall,label,Label_Num
0,20.879744,82.002744,6.502985,202.935536,rice,0
1,21.770462,80.319644,7.038096,226.655537,rice,0
2,23.004459,82.320763,7.840207,263.964248,rice,0
3,26.491096,80.158363,6.980401,242.864034,rice,0
4,20.130175,81.604873,7.628473,262.717340,rice,0
...,...,...,...,...,...,...
2195,26.774637,66.413269,6.780064,177.774507,coffee,21
2196,27.417112,56.636362,6.086922,127.924610,coffee,21
2197,24.131797,67.225123,6.362608,173.322839,coffee,21
2198,26.272418,52.127394,6.758793,127.175293,coffee,21


In [15]:
list_ = {}
for i,j in zip(df['label'].unique(),range(len(list(df['label'].unique())))):
    list_[i] = j

df['Label_Num'] = df['label'].apply(lambda x: list_[x])

In [16]:
df.drop(['label'],axis=1,inplace=True)

In [17]:
# X = df[[col for col in df.columns if col != "Label_Num"]]
X = df[['temperature','humidity','ph','rainfall']]
Y = df['Label_Num']

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.30)

In [19]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

In [20]:
from sklearn.linear_model import LogisticRegression,Lasso,Ridge,ElasticNet
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,mean_absolute_error

models = {
    "Linear Classification" : LogisticRegression(),
    "SVC" : SVC(),
    "K Nearest" : KNeighborsClassifier(),
    "Decision Tree" : DecisionTreeClassifier(),
    "Ada Boost" : AdaBoostClassifier(),
    "Gradient" : GradientBoostingClassifier(),
    "XGB" : XGBClassifier()
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,Y_train)
    Y_pred = model.predict(X_test)

    acc = accuracy_score(Y_test,Y_pred)
    mae = mean_absolute_error(Y_test,Y_pred)
    print(f"R2 value for {list(models.keys())[i]} : ", acc*100)
    print(f"MAE value for {list(models.keys())[i]} : ", mae)
    print("-"*50) # Just to make a boundary

R2 value for Linear Classification :  80.0
MAE value for Linear Classification :  1.4666666666666666
--------------------------------------------------
R2 value for SVC :  90.15151515151516
MAE value for SVC :  0.8636363636363636
--------------------------------------------------
R2 value for K Nearest :  88.93939393939394
MAE value for K Nearest :  0.8984848484848484
--------------------------------------------------
R2 value for Decision Tree :  93.18181818181817
MAE value for Decision Tree :  0.5181818181818182
--------------------------------------------------


c:\Users\kunsh\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


R2 value for Ada Boost :  11.363636363636363
MAE value for Ada Boost :  5.989393939393939
--------------------------------------------------
R2 value for Gradient :  94.0909090909091
MAE value for Gradient :  0.4909090909090909
--------------------------------------------------
R2 value for XGB :  93.18181818181817
MAE value for XGB :  0.47424242424242424
--------------------------------------------------


In [21]:
model = GradientBoostingClassifier()
model.fit(X_train,Y_train)

GradientBoostingClassifier()

In [22]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, Y, cv=5, scoring='r2')

print("R² scores from each fold:", scores)
print("Average R² score:", round(np.mean(scores)*100, 2), "%")

R² scores from each fold: [0.94929418 0.84297007 0.88610954 0.85183512 0.82885375]
Average R² score: 87.18 %


In [24]:
Prediction = model.predict(scale.transform([[31.15 , 74.12,7.7, 677.66 ]]))
for crop,num in list_.items():
    if num == Prediction:
        print(f"Predicted Crop is : {crop.upper()}")
        break

Predicted Crop is : RICE


c:\Users\kunsh\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [26]:
import pickle

In [28]:
with open('model_dict.pkl', 'wb') as f:
    pickle.dump(list_, f)

In [27]:
with open("model_pred_model.pkl","wb") as f:
    pickle.dump(model,f)

with open("model_pred_scaler.pkl","wb") as f:
    pickle.dump(scale,f)